# 多输入多输出通道
:label:`sec_channels`

虽然我们在 :numref:`subsec_why-conv-channels`中描述了构成每个图像的多个通道和多层卷积层。例如彩色图像具有标准的RGB通道来代表红、绿和蓝。
但是到目前为止，我们仅展示了单个输入和单个输出通道的简化例子。
这使得我们可以将输入、卷积核和输出看作二维张量。

当我们添加通道时，我们的输入和隐藏的表示都变成了三维张量。例如，每个RGB输入图像具有$3\times h\times w$的形状。我们将这个大小为$3$的轴称为*通道*（channel）维度。本节将更深入地研究具有多输入和多输出通道的卷积核。

## 多输入通道

当输入包含多个通道时(channel_input)，需要构造一个与输入数据具有相同输入通道数的卷积核，以便与输入数据进行互相关运算。假设输入的通道数为$c_i$，那么卷积核的输入通道数也需要为$c_i$。如果卷积核的窗口形状是$k_h\times k_w$，那么当$c_i=1$时，我们可以把卷积核看作形状为$k_h\times k_w$的二维张量。

然而，当$c_i>1$时，我们卷积核的每个输入通道将包含形状为$k_h\times k_w$的张量。将这些张量$c_i$连结在一起可以得到形状为$c_i\times k_h\times k_w$的卷积核。由于输入和卷积核都有$c_i$个通道，我们可以对每个通道输入的二维张量和卷积核的二维张量进行互相关运算，再对通道求和（将$c_i$的结果相加）得到二维张量。这是多通道输入和多输入通道卷积核之间进行二维互相关运算的结果。

在 :numref:`fig_conv_multi_in`中，我们演示了一个具有两个输入通道的二维互相关运算的示例。阴影部分是第一个输出元素以及用于计算这个输出的输入和核张量元素：

$$\begin{aligned}
\text{channel 0} \quad &(1\times1+2\times2+4\times3+5\times4)+\\
\text{channel 1} \quad &(0\times0+1\times1+3\times2+4\times3)\\
& =56
\end{aligned}
$$。

![两个输入通道的互相关计算。](../img/conv-multi-in.svg)
:label:`fig_conv_multi_in`

为了加深理解，我们(**实现一下多输入通道互相关运算**)。
简而言之，我们所做的就是对每个通道执行互相关操作，然后将结果相加。


In [16]:
import torch
from myd2l import torch as d2l

In [17]:
def corr2d_multi_in(X, K):
    # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

我们可以构造与 :numref:`fig_conv_multi_in`中的值相对应的输入张量`X`和核张量`K`，以(**验证互相关运算的输出**)。


In [18]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

## 多输出通道

到目前为止，不论有多少输入通道，我们还只有一个输出通道。然而，正如我们在 :numref:`subsec_why-conv-channels`中所讨论的，*每一层有多个输出通道是至关重要的*。在最流行的神经网络架构中，随着神经网络层数的加深，我们常会*增加输出通道的维数*，**通过减少空间分辨率以获得更大的通道深度**。

>直观地说，我们可以将每个通道看作对不同特征的响应。而现实可能更为复杂一些，因为每个通道不是独立学习的，而是为了共同使用而优化的。因此，多输出通道并不仅是学习多个单通道的检测器。

用$c_i$和$c_o$分别表示输入和输出通道的数目，并让$k_h$和$k_w$为卷积核的高度和宽度。为了获得多个通道的输出，我们可以**为每个输出通道创建一个形状为$c_i\times k_h\times k_w$的卷积核张量**，这样卷积核的形状是$c_o\times c_i\times k_h\times k_w$。在互相关运算中，每个输出通道先获取所有输入通道，再以对应该输出通道的卷积核计算出结果。

如下所示，我们实现一个[**计算多个通道的输出的互相关函数**]。


In [19]:
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

📦 `torch.stack` 的作用

✅ 功能：
- **将多个相同形状的张量，沿一个**新维度**拼接成一个更高维的张量**
- 所有输入张量必须**形状完全相同**

📐 在你代码中的具体行为：

| 步骤 | 说明 |
|------|------|
| `[corr2d_multi_in(X, k) for k in K]` | 得到 `C_out` 个张量，每个形状为 `(H_out, W_out)` |
| `torch.stack(..., dim=0)` | 沿**第 0 维**堆叠 → 输出形状 `(C_out, H_out, W_out)` |

> 💡 这正是卷积层的**标准输出格式**！

---

 🧪 举个具体例子

假设：
- 输入 `X`: `(3, 6, 6)` （3 通道，6×6 图像）
- 卷积核 `K`: `(2, 3, 3, 3)` （2 个输出通道，每个核 3×3×3）

执行过程：
1. `k0 = K[0]` → shape `(3, 3, 3)`  
   → `corr2d_multi_in(X, k0)` → 输出 `out0: (4, 4)`
2. `k1 = K[1]` → shape `(3, 3, 3)`  
   → `corr2d_multi_in(X, k1)` → 输出 `out1: (4, 4)`
3. `outputs = [out0, out1]` → 两个 `(4,4)` 张量
4. `torch.stack(outputs, dim=0)` → `(2, 4, 4)`

✅ 最终输出：2 个通道的特征图，符合 `nn.Conv2d(3, 2, kernel_size=3)` 的输出。

---

❓ `torch.stack` 的参数详解

```python
torch.stack(tensors, dim=0)
```

| 参数 | 含义 |
|------|------|
| `tensors` | 一个**可迭代对象**（如 list、tuple），包含 N 个**形状相同的张量** |
| `dim` | **插入新维度的位置**（0 ≤ dim ≤ 张量原维度数）• `dim=0`：新维度在最前• `dim=-1`：新维度在最后 |

示例对比：
```python
a = torch.tensor([1, 2])
b = torch.tensor([3, 4])

torch.stack([a, b], dim=0)  # shape (2, 2): [[1,2], [3,4]]
torch.stack([a, b], dim=1)  # shape (2, 2): [[1,3], [2,4]]
```

> ⚠️ 注意：`stack` 和 `cat` 不同！  
> - `stack` **创建新维度**  
> - `cat` **沿现有维度拼接**

通过将核张量`K`与`K+1`（`K`中每个元素加$1$）和`K+2`连接起来，构造了一个具有$3$个输出通道的卷积核。


In [20]:

K = torch.stack((K, K + 1, K + 2), 0)
#print(f'{K=}')
K.shape

torch.Size([3, 2, 2, 2])

下面，我们对输入张量`X`与卷积核张量`K`执行互相关运算。现在的输出包含$3$个通道，第一个通道的结果与先前输入张量`X`和多输入单输出通道的结果一致。


In [21]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

## $1\times 1$ 卷积层

[~~1x1卷积~~]

$1 \times 1$卷积，即$k_h = k_w = 1$，看起来似乎没有多大意义。
毕竟，卷积的本质是有效提取相邻像素间的相关特征，而$1 \times 1$卷积显然没有此作用。
尽管如此，$1 \times 1$仍然十分流行，经常包含在复杂深层网络的设计中。下面，让我们详细地解读一下它的实际作用。

因为使用了最小窗口，$1\times 1$卷积失去了卷积层的特有能力——在高度和宽度维度上，识别相邻元素间相互作用的能力。
其实$1\times 1$卷积的唯一计算发生在通道上。

 :numref:`fig_conv_1x1`展示了使用$1\times 1$卷积核与$3$个输入通道和$2$个输出通道的互相关计算。
这里输入和输出具有相同的高度和宽度，输出中的每个元素都是从输入图像中同一位置的元素的线性组合。
我们可以将$1\times 1$卷积层看作在每个像素位置应用的全连接层，以$c_i$个输入值转换为$c_o$个输出值。
因为这仍然是一个卷积层，所以跨像素的权重是一致的。
同时，$1\times 1$卷积层需要的权重维度为$c_o\times c_i$，再额外加上一个偏置。

![互相关计算使用了具有3个输入通道和2个输出通道的 $1\times 1$ 卷积核。其中，输入和输出具有相同的高度和宽度。](../img/conv-1x1.svg)
:label:`fig_conv_1x1`


上图中，
- 输入层的维度是$(3 \times 3 \times 3 )$,对应(channels_input $\times$ height $\times$ width)
- 权重系数的维度是$(2 \times 3 \times 1 \times 1 )$,对应(channels_output $\times$ channels_input $\times$ kernel_height $\times$ kernel_width)

下面，我们使用全连接层实现$1 \times 1$卷积。
请注意，我们需要对输入和输出的数据形状进行调整。


In [23]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

当执行$1\times 1$卷积运算时，上述函数相当于先前实现的互相关函数`corr2d_multi_in_out`。让我们用一些样本数据来验证这一点。


In [24]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

In [25]:
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6

## 小结

* 多输入多输出通道可以用来扩展卷积层的模型。
* 当以每像素为基础应用时，$1\times 1$卷积层相当于全连接层。
* $1\times 1$卷积层通常用于调整网络层的通道数量和控制模型复杂性。

## 练习

1. 假设我们有两个卷积核，大小分别为$k_1$和$k_2$（中间没有非线性激活函数）。
    1. 证明运算可以用单次卷积来表示。
    1. 这个等效的单个卷积核的维数是多少呢？
    1. 反之亦然吗？
1. 假设输入为$c_i\times h\times w$，卷积核大小为$c_o\times c_i\times k_h\times k_w$，填充为$(p_h, p_w)$，步幅为$(s_h, s_w)$。
    1. 前向传播的计算成本（乘法和加法）是多少？
    1. 内存占用是多少？
    1. 反向传播的内存占用是多少？
    1. 反向传播的计算成本是多少？
1. 如果我们将输入通道$c_i$和输出通道$c_o$的数量加倍，计算数量会增加多少？如果我们把填充数量翻一番会怎么样？
1. 如果卷积核的高度和宽度是$k_h=k_w=1$，前向传播的计算复杂度是多少？
1. 本节最后一个示例中的变量`Y1`和`Y2`是否完全相同？为什么？
1. 当卷积窗口不是$1\times 1$时，如何使用矩阵乘法实现卷积？


[Discussions](https://discuss.d2l.ai/t/1854)


1. 假设我们有两个卷积核，大小分别为$k_1$和$k_2$（中间没有非线性激活函数）。
    1. 证明运算可以用单次卷积来表示。
    1. 这个等效的单个卷积核的维数是多少呢？
    1. 反之亦然吗？

---
 **1.1 证明两个连续卷积（无非线性）可等效为单次卷积**

考虑两个卷积层串联（无激活函数）：

- 第一层：卷积核 $K^{(1)} \in \mathbb{R}^{c_m \times c_i \times k_1 \times k_1}$
- 第二层：卷积核 $K^{(2)} \in \mathbb{R}^{c_o \times c_m \times k_2 \times k_2}$

输入 $X \in \mathbb{R}^{c_i \times h \times w}$

前向传播：
$$
Y = K^{(2)} * (K^{(1)} * X)
$$

由于**卷积满足结合律**（在适当边界条件下，如零填充一致），且**线性操作可交换顺序**，整个系统是**线性时不变（LTI）系统**，因此存在一个等效卷积核 $K_{\text{eq}}$ 使得：
$$
Y = K_{\text{eq}} * X
$$

> ✅ **证明思路**：  
> 在变换域中，卷积变为乘法：  
> $$
> \mathcal{F}(Y) = \mathcal{F}(K^{(2)}) \cdot \mathcal{F}(K^{(1)}) \cdot \mathcal{F}(X) = \mathcal{F}(K_{\text{eq}}) \cdot \mathcal{F}(X)
> $$  
> 所以 $K_{\text{eq}} = \mathcal{F}^{-1}\left( \mathcal{F}(K^{(2)}) \cdot \mathcal{F}(K^{(1)}) \right)$，即 $K^{(2)}$ 与 $K^{(1)}$ 的**互相关（或卷积）**。

在空间域，等效核是两个核的**二维卷积**（注意通道维度需匹配）：
$$
K_{\text{eq}} = K^{(2)} \star K^{(1)}
$$
其中 $\star$ 表示跨通道的卷积组合。

> ⚠️ 注意：这仅在**无非线性、无 stride/padding 不一致**时成立。


**1.2 等效卷积核的尺寸是多少？**

- $K^{(1)}$ 空间尺寸：$k_1 \times k_1$
- $K^{(2)}$ 空间尺寸：$k_2 \times k_2$

两个卷积串联后的**感受野**为：
$$
k_{\text{eq}} = k_1 + k_2 - 1
$$

> ✅ **等效核的空间尺寸为 $(k_1 + k_2 - 1) \times (k_1 + k_2 - 1)$**  
> （这是两个核卷积后的支撑集大小）

通道维度：
- 输入通道：$c_i$
- 输出通道：$c_o$
- 所以 $K_{\text{eq}} \in \mathbb{R}^{c_o \times c_i \times (k_1 + k_2 - 1) \times (k_1 + k_2 - 1)}$


**1.3 反之亦然吗？（单个卷积能否分解为两个小卷积？）**


- **可以的情况**：如果等效核 $K_{\text{eq}}$ 是**低秩的**或具有**可分离结构**（如 separable convolution），则可分解。
  - 例如：$3\times3$ 核若可分离为 $3\times1$ 和 $1\times3$，则可分解。
- **一般情况**：任意 $K_{\text{eq}}$ **不能**精确分解为两个更小卷积核的串联。

> ❌ **反向不成立**：不是所有大卷积核都能等价分解为两个小卷积核（除非满足特定数学条件）。


1. 假设输入为$c_i\times h\times w$，卷积核大小为$c_o\times c_i\times k_h\times k_w$，填充为$(p_h, p_w)$，步幅为$(s_h, s_w)$。
    1. 前向传播的计算成本（乘法和加法）是多少？
    1. 内存占用是多少？
    1. 反向传播的内存占用是多少？
    1. 反向传播的计算成本是多少？

---
**问题 2：卷积的计算与内存成本**

给定：
- 输入：$X \in \mathbb{R}^{c_i \times h \times w}$
- 卷积核：$K \in \mathbb{R}^{c_o \times c_i \times k_h \times k_w}$
- 填充：$(p_h, p_w)$
- 步幅：$(s_h, s_w)$

输出尺寸：
$$
h_o = \left\lfloor \frac{h + 2p_h - k_h}{s_h} \right\rfloor + 1,\quad
w_o = \left\lfloor \frac{w + 2p_w - k_w}{s_w} \right\rfloor + 1
$$

**2.1 前向传播计算成本（乘法和加法）**

对**每个输出元素**：
- 需要 $c_i \times k_h \times k_w$ 次乘法
- 加法次数 ≈ 乘法次数（实际少 1，但通常忽略）

总输出元素数：$c_o \times h_o \times w_o$

✅ **总乘法次数**：
$$
\boxed{c_o \cdot h_o \cdot w_o \cdot c_i \cdot k_h \cdot k_w}
$$

✅ **总加法次数**：近似相同（略少，但同阶）


**2.2 前向传播内存占用**

- 输入：$c_i h_i w_i$
- 卷积核：$c_o c_i k_h k_w$
- 输出：$c_o h_o w_o$

✅ **峰值内存** ≈ 输入 + 核 + 输出（假设无中间缓存优化）  
但通常指**模型参数 + 激活值**：

- **参数内存**：$c_o c_i k_h k_w$（float32 则 ×4 字节）
- **激活内存（前向）**：$\max(c_i h w, c_o h_o w_o)$

> 📌 通常“内存占用”指**激活值（activations）**，即输出张量大小：$\boxed{c_o h_o w_o}$


**2.3 反向传播的内存占用**

反向传播需要保存**前向的输入和输出**用于梯度计算：

- 保存输入 $X$：$c_i h_i w_i$
- 保存输出 $Y$：$c_o h_o w_o$
- （有时还需中间结果）

✅ **反向内存占用** ≈ $\boxed{c_i h w + c_o h_o w_o}$

> 💡 这就是为什么训练比推理更耗显存！

**2.4 反向传播的计算成本**

反向传播需计算两类梯度：


(a) 对输入的梯度 $\frac{\partial L}{\partial X}$
- 相当于用**翻转的卷积核**对 $\frac{\partial L}{\partial Y}$ 做 full 卷积
- 计算量 ≈ $c_i h w \cdot c_o k_h k_w$

(b) 对权重的梯度 $\frac{\partial L}{\partial K}$
- 相当于 $X$ 与 $\frac{\partial L}{\partial Y}$ 的互相关
- 计算量 ≈ $c_o c_i k_h k_w \cdot h_o w_o$

✅ **总反向计算量** ≈ **2 倍前向计算量**（同阶）

$$
\boxed{\sim 2 \cdot c_o h_o w_o c_i k_h k_w}
$$

1. 如果我们将输入通道$c_i$和输出通道$c_o$的数量加倍，计算数量会增加多少？如果我们把填充数量翻一番会怎么样？

---
**3.1 将 $c_i$ 和 $c_o$ 加倍**

前向计算量：
$$
\text{原} \propto c_o c_i \quad \Rightarrow \quad \text{新} \propto (2c_o)(2c_i) = 4 c_o c_i
$$

✅ **计算量变为 4 倍**

> 💡 因为计算量正比于 $c_i \times c_o$

---

**3.2 将填充 $(p_h, p_w)$ 翻倍**

填充**不影响计算量**！  
因为：
- 计算量取决于**输出元素数** $h_o w_o$
- 而 $h_o = \left\lfloor \frac{h + 2p_h - k_h}{s_h} \right\rfloor + 1$

→ 若 $p$ 增加但 $h_o$ 不变（例如 padding 不改变有效输出尺寸），则计算量不变。  
→ 若 padding 导致 $h_o$ 增加（如从 valid 变为 same），则计算量增加。

✅ **结论**：**填充本身不直接增加计算，但可能通过增大输出尺寸间接增加计算量。**

1. 如果卷积核的高度和宽度是$k_h=k_w=1$，前向传播的计算复杂度是多少？

---

当 $k_h = k_w = 1$：

- 每个输出点只需 $c_i$ 次乘法（无空间滑动）
- 输出尺寸：$h_o = h, w_o = w$（若 stride=1, padding=0）

✅ **前向计算量**：
$$
c_o \cdot h \cdot w \cdot c_i \cdot 1 \cdot 1 = \boxed{c_o c_i h w}
$$

> 💡 这正是 **pointwise convolution**，常用于通道变换（如 MobileNet）。



1. 本节最后一个示例中的变量`Y1`和`Y2`是否完全相同？为什么？

---
相同的

1. 当卷积窗口不是$1\times 1$时，如何使用矩阵乘法实现卷积？

---

这是 **im2col（image to column）** 技术。

步骤：

1. **将输入 $X$ 重排为矩阵 $\mathbf{X}_{\text{col}}$**
   - 每一列对应一个卷积窗口的展开（含所有通道）
   - 矩阵形状：$(c_i k_h k_w, h_o w_o)$

2. **将卷积核 $K$ 重排为矩阵 $\mathbf{W}$**
   - 每一行对应一个输出通道的核展开
   - 矩阵形状：$(c_o, c_i k_h k_w)$

3. **矩阵乘法**：
   $$
   \mathbf{Y} = \mathbf{W} \cdot \mathbf{X}_{\text{col}} \quad \Rightarrow \quad \text{shape: } (c_o, h_o w_o)
   $$

4. **reshape 为输出张量**：$(c_o, h_o, w_o)$

✅ **优点**：利用高度优化的 GEMM（通用矩阵乘）库（如 BLAS）
❌ **缺点**：内存膨胀（im2col 需要额外存储）

> 📌 这是 cuDNN、MKL 等底层库实现卷积的标准方法。
